# Python Class for Web Scrapping

In [1]:
from python_script.web_scrapping import Word

# Extract the Lexicon of a Word

In [2]:
w1 = Word('maksud')
w1.extractLexicon()
w1.getLexicon()

['maksud',
 6,
 'kata nama',
 ['1. tujuan, hajat, kehendak, niat',
  '2. erti yg tersimpul dlm sesuatu ayat (perbuatan dll)'],
 ['1. maksud aku ke sini ialah hendak berjumpa dgn engkau',
  '2. apakah maksud perkataanmu itu?'],
 ['tujuan',
  'hajat',
  'hasrat',
  'niat',
  'motif',
  'matlamat',
  'objektif',
  'keinginan',
  'kemahuan',
  'kehendak',
  'murad',
  'kasad',
  'anju',
  'hujung'],
 ['bermaksud', 'memaksudkan']]

# Create a List to Store Lexicons

In [3]:
words = []
words.append(w1.getLexicon())

## Expand Lexicons Size

In [4]:
import re
import string

prepared_keyword = w1.getDefinitionStr()
prepared_keyword = prepared_keyword.replace('\xad', ' ').replace('\xa0', ' ').replace('/', ' ')
prepared_keyword = prepared_keyword.translate(str.maketrans('', '', string.punctuation))
prepared_keyword = re.sub(r"\d+", "", prepared_keyword)

prepared_keyword = prepared_keyword.split(' ')
while '' in prepared_keyword:
    prepared_keyword.remove('')
    
print(prepared_keyword)

['ma', 'sud', 'maksud', 'Definisi', 'tujuan', 'hajat', 'kehendak', 'niat', 'aku', 'ke', 'sini', 'ialah', 'hendak', 'berjumpa', 'dgn', 'engkau', 'hatiku', 'pun', 'hendak', 'ke', 'sana', 'juga', 'erti', 'yg', 'tersimpul', 'dlm', 'sesuatu', 'ayat', 'per', 'buatan', 'dll', 'apakah', 'perkataanmu', 'itu', 'aku', 'masih', 'belum', 'tahu', 'lagi', 'aku', 'dipanggil', 'bermaksud', 'bertujuan', 'berhajat', 'berke', 'hendak', 'berniat', 'hendak', 'kami', 'hendak', 'menyerahkan', 'wang', 'itu', 'dgn', 'seberapa', 'segera', 'mengandungi', 'erti', 'apa', 'yg', 'diucapkan', 'sebentar', 'tadi', 'itu', 'tidak', 'demikian', 'memaksudkan', 'menghendaki', 'meminta', 'dialah', 'yg', 'dimaksudkan', 'hadir', 'dlm', 'seminar', 'itu', 'memberi', 'erti', 'yg', 'sebenar', 'hendak', 'menyampaikan', 'atau', 'menyatakan', 'sbg', 'mak', 'sud', 'Ali', 'masih', 'samarsamar', 'tentang', 'apa', 'yg', 'dimaksudkan', 'oleh', 'perempuan', 'tua', 'itu', 'apabila', 'ia', 'menerangkan', 'tentang', 'pendekatan', 'struktur', '

In [5]:
for word in prepared_keyword:
    w = Word(word)
    w.extractLexicon()

    words.append(w.getLexicon())

for word in ['tenaga', 'jelas', 'dunia', 'negara', 'tanah', 'asas']:
    w = Word(word)
    w.extractLexicon()
    prepared_keyword_temp = w.getDefinitionStr()
    prepared_keyword_temp = prepared_keyword_temp.replace('\xad', ' ').replace('\xa0', ' ').replace('/', ' ')
    prepared_keyword_temp = prepared_keyword_temp.translate(str.maketrans('', '', string.punctuation))
    prepared_keyword_temp = re.sub(r"\d+", "", prepared_keyword_temp)

    prepared_keyword_temp = prepared_keyword_temp.split(' ')
    while '' in prepared_keyword_temp:
        prepared_keyword_temp.remove('')
    
    for word_1 in prepared_keyword_temp:
        w1 = Word(word_1)
        w1.extractLexicon()
    
        words.append(w1.getLexicon())

In [6]:
print(len(words))

930


There are 930 non-cleansed records by scrapping information from 7 Malay vocabularies.

# Data Processing by PySpark

## Create Spark Session

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

24/12/14 12:11:57 WARN Utils: Your hostname, JORDAN. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/14 12:11:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/14 12:11:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Defines Columns' Heading and Create DataFrame

In [8]:
columns = ['vocab', 'word_length', 'part_of_speech', 'definitions', 'sentences_eg', 'synonyms', 'derived_words']
df = spark.createDataFrame(words, columns)

In [9]:
df.show()

+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|   vocab|word_length|part_of_speech|         definitions|        sentences_eg|            synonyms|       derived_words|
+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|  maksud|          6|     kata nama|[1. tujuan, hajat...|[1. maksud aku ke...|[tujuan, hajat, h...|[bermaksud, memak...|
|      ma|          2|              |                  []|                  []|                  []|                  []|
|     sud|          3|              |                  []|                  []|                  []|                  []|
|  maksud|          6|     kata nama|[1. tujuan, hajat...|[1. maksud aku ke...|[tujuan, hajat, h...|[bermaksud, memak...|
|Definisi|          8|              |                  []|                  []|                  []|                  []|
|  tujuan|          6|  

## Transform All Invalid Datatype to Better Form

In [10]:
from pyspark.sql.functions import col, concat_ws

for cols in ['definitions', 'sentences_eg', 'synonyms', 'derived_words']:
    df = df.withColumn(cols, concat_ws(";", col(cols)))

df.show()

+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|   vocab|word_length|part_of_speech|         definitions|        sentences_eg|            synonyms|       derived_words|
+--------+-----------+--------------+--------------------+--------------------+--------------------+--------------------+
|  maksud|          6|     kata nama|1. tujuan, hajat,...|1. maksud aku ke ...|tujuan;hajat;hasr...|bermaksud;memaksu...|
|      ma|          2|              |                    |                    |                    |                    |
|     sud|          3|              |                    |                    |                    |                    |
|  maksud|          6|     kata nama|1. tujuan, hajat,...|1. maksud aku ke ...|tujuan;hajat;hasr...|bermaksud;memaksu...|
|Definisi|          8|              |                    |                    |                    |                    |
|  tujuan|          6|  

## Store the DataFrame in the form of CSV

In [11]:
df.coalesce(1).write.csv('lexicon_data', header=True, mode='overwrite')

In [12]:
from python_script.hdfs_commander import *

run_hdfs_command('hdfs dfs -mv lexicon_data/part-00000-*.csv lexicon_data/non_processed_data.csv')
run_hdfs_command('hdfs dfs -get lexicon_data/non_processed_data.csv lexicon_data/non_processed_data.csv')

## Terminate Spark Session

In [13]:
spark.stop()